In [29]:
# import statements
from time import sleep
from kafka import KafkaConsumer
from kafka import KafkaProducer
import time 
import json

topic = 'input_topic'

# connecting to kafka consumer
def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=10000,
                                   auto_offset_reset='earliest',
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer

# connecting to kafka producer
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0,10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
# publish aggregated payload to output_topic
def publish_to_output_topic(producer_instance, output, key):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(output, encoding='utf-8')
        producer_instance.send('output_topic', key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(output))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

# aggregate the data consumed from producer        
def aggregate(data):
    payload = {
        'userId': data['userId'],
        'firstSeen': data['metadata']['sentAt'],
        'lastSeen': int(time.time())
    }
    return payload

# consume the message and publish it to output_topic
def consume_messages(consumer):
    try:
        for message in consumer:
            data = json.loads(message.value.decode('utf-8'))
            output = aggregate(data)
            producer = connect_kafka_producer()
            data = json.dumps(output)
            publish_to_output_topic(producer, data, output['userId'])
    except Exception as ex:
        print(str(ex))
    
    
if __name__ == '__main__':
    consumer = connect_kafka_consumer()
    consume_messages(consumer)
    
    

Message published successfully. Data: {"userId": "j11288090", "firstSeen": 1534382478, "lastSeen": 1631603175}
Message published successfully. Data: {"userId": "j11288090", "firstSeen": 1534382478, "lastSeen": 1631603175}
Message published successfully. Data: {"userId": "j11288090", "firstSeen": 1534382478, "lastSeen": 1631603175}
